# Visualize Toy Networks 
This script produces visualizations for three toy models (cayley tree, cube lattice and torus lattice) 
to provide an overview of all included layouts in the cartoGRAPHs project. 

In [1]:
from cartoGRAPHs import * 

from func_load_data import *
from func_visual_properties import * 
from func_calculations import * 
from func_embed_plot import * 
from func_exportVR import * 

----
# CHOOSE A NETWORK
----

In [2]:
# CAYLEY TREE

organism = 'tree'
n = 1093
r = 3
G = nx.full_rary_tree(r,n)

In [3]:
# CUBE LATTICE 

organism = 'grid'
G = nx.grid_graph([10,10,10],periodic=False)
# converting node labels to integers 
start = 1
G = nx.convert_node_labels_to_integers(G,first_label=start)

In [ ]:
# TORUS LATTICE 

organism = 'torus'
G = nx.grid_2d_graph(44,22,periodic=True)
# converting node labels to integers 
start = 1
G = nx.convert_node_labels_to_integers(G,first_label=start)

-----
# COLOR SETTINGS
-----

In [4]:
opacity_nodes = 0.9
node_edge_col = '#696969' 

gray_colors = '#696969'

closeness = nx.closeness_centrality(G)
d_node_colors = {}
for node, cl in sorted(closeness.items(), key = lambda x: x[1], reverse = 1):
    d_node_colors[node] = round(cl,4)
        
col_pal = 'viridis'
d_colors = color_nodes_from_dict(G, d_node_colors, palette = col_pal)
colors = list(d_colors.values())
edge_color = '#d3d3d3'

size = 70
size3d = 2 #3.5

#################
# E D G E S
#################
edge_width = 2.0
edge_color = '#999999' #'#696969' 
opacity_edges = 0.3

In [6]:
layoutmethod = 'metapathvec'
dim = 3
n_n = 8
spr = 0.9
md = 0.0
metr = 'cosine'

if layoutmethod == 'local':
    posG = layout_local_umap(G,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 

elif layoutmethod == 'global':
    posG = layout_global_umap(G,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 

elif layoutmethod == 'nodevec':
    posG = layout_nodevec_umap(G,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 
    
elif layoutmethod == 'importance':
    posG = layout_importance_umap(G,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 
    
elif layoutmethod == 'graphwave':
    posG = layout_graphwave_umap(G,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 
    
elif layoutmethod == 'metapathvec':
    posG = layout_metapathvec_umap(G,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 
    
elif layoutmethod == 'functional':
    scale = 1
    val = 0
    rows = len(list(G.nodes()))
    feat_one = [(val) if i%3 else (scale) for i in range(rows)]
    feat_two = [(val) if i%2 or feat_one[i]==scale in feat_one else (scale) for i in range(rows)]
    feat_three = [(scale) if feat_one[i]==val and feat_two[i]==val and i not in feat_one and i not in feat_two else val for i in range(rows)]
    feat_matrix = np.vstack((feat_one,feat_two,feat_three))
    
    DF_functional = pd.DataFrame(feat_matrix)
    DF_functional.index = ['100','101','102']
    DF_functional = DF_functional.T
    DF_functional.index = list(G.nodes())

    posG = layout_functional_umap(G,DF_functional,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 
    
elif layoutmethod == 'attrivec':
    scale = 1
    val = 0
    rows = len(list(G.nodes()))
    feat_one = [(val) if i%3 else (scale) for i in range(rows)]
    feat_two = [(val) if i%2 or feat_one[i]==scale in feat_one else (scale) for i in range(rows)]
    feat_three = [(scale) if feat_one[i]==val and feat_two[i]==val and i not in feat_one and i not in feat_two else val for i in range(rows)]

    DF_functional = pd.DataFrame(feat_matrix)
    DF_functional.index = ['100','101','102']
    DF_functional = DF_functional.T
    DF_functional.index = list(G.nodes())
    
    posG = layout_attrivec_umap(G,DF_functional,dim,n_neighbors=n_n, spread=spr, min_dist=md, metric=metr) 

In [7]:
if dim == 2:
    plt.figure(figsize=(25,25))

    nx.draw_networkx_nodes(G, posG, node_color = colors, node_size= size, alpha = opacity_nodes)
    nx.draw_networkx_edges(G, posG, width = edge_width, edge_color = edge_color, alpha = opacity_edges*0.15)

    plt.box(False)

    plt.savefig('output_plots/benchmark/2DPortrait_'+organism+'_'+layoutmethod+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_metr'+metr+'.pdf')
    plt.show()
    
elif dim == 3: 
    
    umap_nodes = get_trace_nodes_3D(posG, None, colors, size3d, 
                                    opacity_nodes)
    umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.15, linewidth = 0.75)
    data = [umap_nodes,umap_edges]

    path = 'output_plots/benchmark/'
    fname = '3Dportrait_'+organism+'_'+layoutmethod+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_metr'+metr

    plot_3D(data,path,fname, 'light')